In [1]:
import pandas as pd
import numpy as np
import re

from elder_care.config import BLD
from elder_care.config import SRC

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
data = pd.read_csv(BLD / "data" / "estimation_data.csv")

In [4]:
data.shape

(6545, 400)

In [5]:
data[["mergeid", "care", "lagged_care"]].head(20)

mergeid  care  lagged_care
0   DE-000132-01     0          NaN
1   DE-000132-01     0          0.0
2   DE-000132-01     0          0.0
3   DE-000132-01     0          0.0
4   DE-000132-01     0          0.0
5   DE-000132-01     0          0.0
6   DE-000132-01     0          0.0
7   DE-000554-01     0          NaN
8   DE-000554-01     0          0.0
9   DE-000554-01     0          0.0
10  DE-000554-01     0          0.0
11  DE-001237-02     0          NaN
12  DE-001237-02     0          0.0
13  DE-001237-02     0          0.0
14  DE-001237-02     0          0.0
15  DE-001350-01     0          NaN
16  DE-001350-01     0          0.0
17  DE-001381-01     0          NaN
18  DE-001381-01     0          0.0
19  DE-001381-01     0          0.0

## Transition from caregiving to non-caregiving

In [6]:
data["care"].unique()

array([0, 1])

In [7]:
dat = dat.sort_values(by=["mergeid", "int_year"], ascending=[True, True])
dat["lagged_care"] = dat.groupby("mergeid")["care"].shift(1)
dat["care_experience"] = dat.groupby("mergeid")["lagged_care"].cumsum()

NameError: name 'dat' is not defined

In [15]:
df = data.copy()

In [16]:
# add an arbitrary counts column for group by
df['counts'] = 1

In [17]:
# group together the combinations then unstack to get matrix
trans_matrix = df.groupby("mergeid")["care", "lagged_care"].count().unstack()

ValueError: Cannot subset columns with a tuple with more than one element. Use a list instead.

In [18]:
trans_matrix = df.groupby(["care", "lagged_care"]).count().unstack()

In [19]:
# max the columns a bit neater
trans_matrix.columns = trans_matrix.columns.droplevel()

In [20]:
trans_matrix

lagged_care   0.0  1.0   0.0  1.0   0.0  1.0   0.0  1.0   0.0  1.0   0.0  1.0  \
care                                                                            
0            3006  384  3006  384  3006  384  3006  384  3006  384  3006  384   
1             315  174   315  174   315  174   315  174   315  174   315  174   

lagged_care   0.0  1.0   0.0  1.0   0.0  1.0   0.0  1.0  0.0  1.0  0.0  1.0  \
care                                                                          
0            3006  384  3006  384  2624  300  2624  300   38    1   29    2   
1             315  174   315  174   315  174   315  174    3    2    2    1   

lagged_care  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0   0.0  1.0   0.0  1.0  \
care                                                                        
0             28    1  191   19   32    3    0    0  1495  259  1269  182   
1              2    1   18    9    6    0    0    0   282  160   170   85   

lagged_care  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  \
care                                                                           
0            751  185  339   61  760   75  861  113    9    0    1    0  716   
1            260  142   90   48   29   18   77   37    1    0    1    0  253   

lagged_care  1.0  0.0  1.0  0.0  1.0   0.0  1.0   0.0  1.0  0.0  1.0  0.0  \
care                                                                        
0            184  327   61  126   34  2165  243  2155  245  705  183  324   
1            141   89   48    4    1   236  137   234  138  244  133   87   

lagged_care  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  \
care                                                                           
0             60   29    2   29    2   29    2   29    2   29    2   29    2   
1             47    2    1    2    1    2    1    2    1    2    1    2    1   

lagged_care  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  \
care                                                                           
0             29    2   29    2   29    2   29    2   29    2   29    2   29   
1              2    1    2    1    2    1    2    1    2    1    2    1    2   

lagged_care  1.0  0.0  1.0  0.0  1.0  0.0  1.0   0.0  1.0   0.0  1.0  0.0  \
care                                                                        
0              2   28    2    9    1   29    2  1013   56  2611  295  805   
1              1    2    1    0    0    2    1   107   69   315  174   74   

lagged_care  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  \
care                                                                           
0             47  805   47  805   47  805   47  805   47  805   47  805   47   
1             57   74   57   74   57   74   57   74   57   74   57   74   57   

lagged_care  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0   0.0  1.0  0.0  1.0  0.0  \
care                                                                            
0            805   47  805   47  805   47  105    8  1900  135  613   53  162   
1             74   57   74   57   74   57   11   10   313  174  303  170  112   

lagged_care  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  \
care                                                                           
0             11   35    1  502   51  127   11   24    1  613   53  162   11   
1             71   30   21  253  155   95   65   21   19  303  170  112   71   

lagged_care  0.0  1.0  0.0  1.0  0.0  1.0   0.0  1.0  0.0  1.0  0.0  1.0  0.0  \
care                                                                            
0             35    1  613   53  162   11  1732  113   22    3   22    3   22   
1             30   21  303  170  112   71   280  142   18   13   18   13   18   

lagged_care  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  \
care                                                                           
0              3

In [21]:
df = df.sort_values(by=['mergeid', 'int_year'])

# Create a transition matrix using crosstab
transition_matrix = pd.crosstab(df['lagged_care'], df['care'], normalize='index')

transition_matrix

care                0         1
lagged_care                    
0.0          0.905149  0.094851
1.0          0.688172  0.311828

In [22]:
# Create a transition matrix using crosstab
transition_matrix = pd.crosstab(df['lagged_care'], df['care'], normalize=False)

transition_matrix

care            0    1
lagged_care           
0.0          3006  315
1.0           384  174

In [26]:
# check
df[(df["lagged_care"] == True) & (df["care"] == True)].shape[0] / df[
    (df["lagged_care"] == True)
].shape[0]

0.3118279569892473